In [12]:
import cv2
import numpy as np
import glob
from skimage import io
from skimage import measure
import imutils
from imutils import contours
from scipy.stats import entropy as en
import pandas as pd
import skimage.measure    
from matplotlib import pyplot as plt

from imutils import contours
from skimage import measure
import numpy as np
import argparse
import imutils

# Exudate Extraction

In [13]:
path=glob.glob("Modified Base\*.tif")
for i in path:
    img=cv2.imread(i,cv2.IMREAD_UNCHANGED)
    img=cv2.resize(img,(500,500))
    green_channel = img[:,:,1]
    green_channel=cv2.resize(green_channel,(500,500))
    complement_of_green_channel=255-green_channel
    
#     optic_disc=green_channel-complement_of_green_channel
#     blank=np.zeros(green_channel.shape[:2],dtype='uint8')
#     ret,threshed_img=cv2.threshold(green_channel,130,255,cv2.THRESH_BINARY)
#     kernel = np.ones((10,10),np.uint8)
#     dilation = cv2.dilate(threshed_img,kernel,iterations = 4)
#     contours,hierarchies=cv2.findContours(dilation,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
#     corted_conted_by_area=sorted(contours,key=cv2.contourArea,reverse=True)
#     blank1=np.zeros(green_channel.shape[:2],dtype='uint8')
#     cv2.drawContours(blank1,corted_conted_by_area,0,(255,255,255),-1)
    
    #Exudate with optic disc
    median=cv2.medianBlur(complement_of_green_channel,43)
    Final_difference=cv2.subtract(median,complement_of_green_channel)
    ret,th=cv2.threshold(Final_difference,25,255,cv2.THRESH_BINARY)
    
    #optic disk
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((2,2),np.uint8)
    diff_green_cyan=cv2.subtract(gray,complement_of_green_channel)
    ret,optic_disc=cv2.threshold(diff_green_cyan,5,255,cv2.THRESH_BINARY)
    optic_disc = cv2.dilate(optic_disc,kernel,iterations = 20)
    contours,hierarchies=cv2.findContours(optic_disc,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    corted_conted_by_area=sorted(contours,key=cv2.contourArea,reverse=True)
    big_disc=np.zeros(green_channel.shape[:2],dtype='uint8')
    cv2.drawContours(big_disc,corted_conted_by_area,0,(255,255,255),-1)


        ###Detecting Blood Vessels




        ###Optic Disc done

        #Image_without_disc=th
    Image_without_disc=cv2.subtract(th,big_disc)

    Image_without_disc=cv2.medianBlur(Image_without_disc,1)
        #Image_without_disc=cv2.dilate(Image_without_disc,kernel,iterations = 2)
    blank_border=np.zeros(Image_without_disc.shape[:2],dtype='uint8')
    axesLength = (150, 238)
    angle = 180
    border = cv2.ellipse(blank_border, (250,250), axesLength,angle, 0, 360, (255,255,255), 40)
    exudate=cv2.subtract(Image_without_disc,border)


    kernel = np.ones((3,3),np.uint8)
    dilated_exudate = cv2.dilate(exudate,kernel,iterations = 1)



    output="Exudates"+"\\"+i
    cv2.imwrite(output,dilated_exudate)
    #cv2.imshow("Images",border1)    
    cv2.waitKey(0) 


# Texture features and defects extraction from retinal image

In [16]:
img_name=[]
sd_red=[]
sd_green=[]
sd_blue=[]
d_o_v=[]
d_o_e=[]
micro_num=[]
green_entropy=[]
for i in range(1,1201):
    img_name.append("image "+str(i)+".tif")
    original_path="Modified Base/image "+str(i)+".tif"
    vessel_path="FinalbloodVessels/Modified Base/image "+str(i)+".tif"
    exudate_path="Exudates/Modified Base/image "+str(i)+".tif"
    micro_path="Microaneurysm/image "+str(i)+".tif"
    RGB=cv2.imread(original_path,cv2.IMREAD_UNCHANGED)
    RGB=cv2.resize(RGB,(800,800))
    VESSEL=cv2.imread(vessel_path,cv2.IMREAD_UNCHANGED)
    VESSEL=cv2.resize(VESSEL,(800,800))
    EXUDATE=cv2.imread(exudate_path,cv2.IMREAD_UNCHANGED)
    EXUDATE=cv2.resize(EXUDATE,(800,800))
    MICRO=cv2.imread(micro_path,cv2.IMREAD_UNCHANGED)
    MICRO=cv2.resize(MICRO,(800,800))

    (R,G,B)=cv2.split(RGB)
    std_red=np.std(R)
    std_green=np.std(G)
    std_blue=np.std(B)
    white_vessel=0
    white_exudate=0
    white_micro=0
    flat_exudate=np.array(EXUDATE).flatten()
    flat_vessel=np.array(VESSEL).flatten()
    flat_micro=np.array(MICRO).flatten()
    for i in range(0,len(flat_exudate)):
        if flat_vessel[i]==255:
            white_vessel=white_vessel+1
        if flat_exudate[i]==255:
            white_exudate=white_exudate+1
        if flat_micro[i]==255:
            white_micro=white_micro+1

    density_of_vessel=white_vessel/len(flat_vessel)
    density_of_exudate=white_exudate/len(flat_exudate)
    #Features
    sd_red.append(std_red)
    sd_green.append(std_green)
    sd_blue.append(std_blue)
    d_o_v.append(white_vessel)
    d_o_e.append(white_exudate)
    micro_num.append(white_micro)
    
    
    RGB_to_gray = cv2.cvtColor(RGB, cv2.COLOR_BGR2GRAY)
    green_entropy.append(skimage.measure.shannon_entropy(RGB_to_gray))
    

In [18]:
dp=pd.read_excel("Modifiedfeatures.xls")
label=dp['Retinopathy grade']

In [19]:
features_and_lebels={
    'IMAGE_NAME':img_name,
    'STD_RED':sd_red,
    'STD_GREEN':sd_green,
    'STD_BLUE':sd_blue,
    'VESSEL_DENSITY':d_o_v,
    'EXUDATE_DENSITY':d_o_e,
    'MICRO_NUM':micro_num,
    'GREEN_entropy':green_entropy,
    'Label':label
}
df=pd.DataFrame(features_and_lebels)
#df.head(-1)
df

,IMAGE_NAME,STD_RED,STD_GREEN,STD_BLUE,VESSEL_DENSITY,EXUDATE_DENSITY,MICRO_NUM,GREEN_entropy,Label
0,image 1.tif,15.317099,45.780791,77.688028,25927,86,16,4.617419,3
1,image 2.tif,9.609404,26.877452,54.475988,23038,639,6,4.393661,0
2,image 3.tif,10.870924,27.995256,53.205162,22561,1207,6,4.186353,1
3,image 4.tif,24.943999,44.902029,106.708809,32954,0,5,4.866068,2
4,image 5.tif,23.708348,45.051394,108.998175,30822,0,30,4.807060,3
...,...,...,...,...,...,...,...,...,...
1194,image 1195.tif,7.854549,34.370738,115.181734,27719,0,8,4.479729,2
1195,image 1196.tif,6.758208,33.362783,81.205014,13823,112,18,4.291761,3
1196,image 1197.tif,7.190850,35.297010,96.347565,22423,1116,2,4.940130,3
1197,image 1198.tif,23.723616,56.699960,119.209698,33011,1321,13,5.183099,3


In [20]:
image_name=[]
std_red=[]
std_green=[]
std_blue=[]
total_vessel=[]
total_exudate=[]
total_micro=[]
entropy_green=[]
targets=[]
count=0
num_of_noret=0
num_of_sevret=0
for i in range(0,1200):
    if df.Label[i]==0 and num_of_noret<250:
        image_name.append(df.IMAGE_NAME[i])
        std_red.append(df.STD_RED[i])
        std_green.append(df.STD_GREEN[i])
        std_blue.append(df.STD_BLUE[i])
        total_vessel.append(df.VESSEL_DENSITY[i])
        total_exudate.append(df.EXUDATE_DENSITY[i])
        total_micro.append(df.MICRO_NUM[i])
        entropy_green.append(df.GREEN_entropy[i])
        targets.append(0)
        num_of_noret=num_of_noret+1
    if df.Label[i]==3 and num_of_sevret<250:
        image_name.append(df.IMAGE_NAME[i])
        std_red.append(df.STD_RED[i])
        std_green.append(df.STD_GREEN[i])
        std_blue.append(df.STD_BLUE[i])
        total_vessel.append(df.VESSEL_DENSITY[i])
        total_exudate.append(df.EXUDATE_DENSITY[i])
        total_micro.append(df.MICRO_NUM[i])
        entropy_green.append(df.GREEN_entropy[i])
        targets.append(1)
        num_of_sevret=num_of_sevret+1
        

In [21]:
features_with_lebels={
    'IMAGE_NAME':image_name,
    'STD_RED':std_red,
    'STD_GREEN':std_green,
    'STD_BLUE':std_blue,
    'VESSEL_DENSITY':total_vessel,
    'EXUDATE_DENSITY':total_exudate,
    'MICRO_NUM':total_micro,
    'GREEN_entropy':entropy_green,
    'Label':targets
}
df=pd.DataFrame(features_with_lebels)
df.head(45)

,IMAGE_NAME,STD_RED,STD_GREEN,STD_BLUE,VESSEL_DENSITY,EXUDATE_DENSITY,MICRO_NUM,GREEN_entropy,Label
0,image 1.tif,15.317099,45.780791,77.688028,25927,86,16,4.617419,1
1,image 2.tif,9.609404,26.877452,54.475988,23038,639,6,4.393661,0
2,image 5.tif,23.708348,45.051394,108.998175,30822,0,30,4.807060,1
3,image 6.tif,11.068267,24.494689,70.856791,24010,757,42,4.402146,0
4,image 7.tif,10.807191,22.985109,64.333279,24288,1239,66,4.298415,0
5,image 8.tif,7.350861,31.579776,65.915293,28561,138,0,4.185515,0
6,image 9.tif,7.734439,33.301360,68.386266,27322,384,6,4.062188,1
7,image 10.tif,11.444244,35.777013,77.006650,23230,33,0,4.277424,0
8,image 11.tif,10.020634,31.450613,68.313884,18971,296,7,3.905422,0
9,image 12.tif,17.277578,38.726045,53.893939,20249,597,28,4.470468,1


In [22]:
from sklearn.utils import shuffle
df = shuffle(df)

In [23]:
df

,IMAGE_NAME,STD_RED,STD_GREEN,STD_BLUE,VESSEL_DENSITY,EXUDATE_DENSITY,MICRO_NUM,GREEN_entropy,Label
314,image 418.tif,4.481753,47.282137,100.236295,11044,164,35,4.509904,1
400,image 528.tif,21.664889,51.106453,92.531291,26455,0,4,4.628310,0
474,image 1091.tif,17.396427,37.014259,79.600979,24144,135,56,4.420312,1
90,image 117.tif,11.755068,46.371232,87.475430,26263,710,61,4.252866,1
207,image 283.tif,15.591712,40.608429,91.500743,7493,49,1,4.296952,0
...,...,...,...,...,...,...,...,...,...
121,image 163.tif,18.765469,30.523662,68.475896,27325,129,19,4.490917,0
477,image 1094.tif,20.749505,43.533885,110.377271,26756,266,35,4.517017,1
389,image 516.tif,12.860502,45.698966,84.838098,24324,0,0,4.388268,0
214,image 292.tif,24.839891,42.303713,100.583363,21781,0,1,4.490582,0


In [24]:
df.to_csv('Allfeatures_and_labels.csv',index=False)